This script takes these as inputs

* City latlon file
* Best-models and calibration-function file
* Scenario-years file
* Historical observed, historical modeled, and future modeled meteorological data stored in Google Earth Engine

...and estimates for each city a set of future indicator values based on 14 climate hazards. The indicators are expected value and probabilities of exceeding given hazard thresholds, both estimated for given global warming scenario years.

In [1]:
import ee
import math
import warnings
import json
warnings.filterwarnings('ignore')
from collections import defaultdict
import numpy as np
import pandas as pd
from scipy import stats
import scipy

import datetime, calendar
import spei

ee.Authenticate()
ee.Initialize()

Enter verification code: 4/1AdLIrYde4WAwSB4dzwVMDzznxZDLh6GGL0ru-GhZfxXRWrRmSNhKFVPGPf8

Successfully saved authorization token.


In [2]:
HIST_START = 1980
HIST_END = 2014

PERCENTILE_STARTYEAR = 1980
PERCENTILE_ENDYEAR = 2019

NUM_BEST_MODELS = 3

In [3]:
# Unit conversions and variable names for NEX-GDDP-CMIP6 and ERA5

VARIABLES = {
    'tas': {
        'era_varname': 'mean_2m_air_temperature',
        'nex_transform': lambda x: x - 273.5,
        'era_transform': lambda x: x - 273.5
    },
    'tasmax': {
        'era_varname': 'maximum_2m_air_temperature',
        'nex_transform': lambda x: x - 273.5,
        'era_transform': lambda x: x - 273.5
    },
    'tasmin': {
        'era_varname': 'minimum_2m_air_temperature',
        'nex_transform': lambda x: x - 273.5,
        'era_transform': lambda x: x - 273.5
    },
    'pr': {
        'era_varname': 'total_precipitation',
        'nex_transform': lambda x: x * 86400,
        'era_transform': lambda x: x * 1000
    },
    'hurs': {
        'era_varname': None,
       'nex_transform': lambda x: x,
        'era_transform': lambda x: x
    },
    'maxwetbulb': {
        'era_varname': None,
       'nex_transform': lambda x: x,
        'era_transform': lambda x: x
    }
}

In [4]:
MODELS = ['MIROC-ES2L', 'UKESM1-0-LL', 'EC-Earth3-Veg-LR', 'GFDL-ESM4', 'INM-CM5-0', 'MRI-ESM2-0', 'FGOALS-g3', 'IPSL-CM6A-LR', 'CanESM5']

In [5]:
# Read city-latlon file. File is header-less CSV. Columns are:
# idnum, UNUSED, latitude, longidude, UNUSED, UNUSED, UNUSED, UNUSED

CITYLATLON = {}
with open('ghsl_500k.csv', 'r') as ifile:
    for line in ifile.readlines():
        items = [i.strip() for i in line.split(',')]
        CITYLATLON['city_{0}'.format(items[0])] = (float(items[2]), float(items[3]), int(items[0]))

In [6]:
def calendardate_percentiles(nex_varname, q, latlon, sh_hem=False):
    # Percentiles based on given calendar date
    hist_start = PERCENTILE_STARTYEAR
    hist_end = PERCENTILE_ENDYEAR
    allyears = []
    for year in range(hist_start, hist_end):
        allyears.append(get_observed_gee(nex_varname, latlon, start_year=year, end_year=year, southern_hem=False))
    if not sh_hem:
        return np.percentile(np.vstack(allyears), q, axis=0)
    else:
        res = np.percentile(np.vstack(allyears), q, axis=0)
        return np.concatenate([res[152:], res[:152]])

def wholeyear_percentile(nex_varname, q, latlon):
    # percentiles based not on one calendar date, but on all dates
    if not nex_varname == 'ari':
        hist_start = PERCENTILE_STARTYEAR
        hist_end = PERCENTILE_ENDYEAR
        allyears = []
        for year in range(hist_start, hist_end):
            allyears.append(get_observed_gee(nex_varname, latlon, start_year=year, end_year=year, southern_hem=False))
        return np.percentile(np.concatenate(allyears).flatten(), q)
    else:
        hist_start = PERCENTILE_STARTYEAR
        hist_end = PERCENTILE_ENDYEAR
        allyears = []
        for year in range(hist_start, hist_end):
            allyears.append(get_observed_gee('pr', latlon, start_year=year, end_year=year, southern_hem=False))
        ari_data = ari(np.concatenate(allyears).flatten())
        return np.percentile(ari_data, 95)

def yearextreme_percentile(nex_varname, q, latlon, wantmax):
    # Percentiles 
    hist_start = PERCENTILE_STARTYEAR
    hist_end = PERCENTILE_ENDYEAR
    allyears = get_observed_gee(nex_varname, latlon, hist_start, hist_end, southern_hem=False)
    return np.percentile(allyears, q)

def d2j(datestring):
    # Date to Julian date
    d = datetime.date.fromisoformat(datestring)
    jday = d.timetuple().tm_yday
    if calendar.isleap(d.year) and jday > 59:
        jday -= 1
    return jday

def removeLeapDays(arr, start_year, end_year, southern_hem):
    indices_to_remove = []
    for year in range(start_year, end_year + 1):
        if calendar.isleap(year):
            indices_to_remove.append(((year-start_year) * 365) + [0,183][int(southern_hem)] + len(indices_to_remove) + 59)
    return np.delete(arr, indices_to_remove)

def count_runs(tf_array, min_runsize):
    # Return number of consecutive runs or at least minimum run length
    # Accepts true-false array with one row per year, returns array of counts -- one count per year
    falses = np.zeros(tf_array.shape[0]).reshape((tf_array.shape[0],1))
    extended_a = np.concatenate([[0], tf_array, [0]])
    df = np.diff(extended_a)
    starts = np.nonzero(df == 1)[0]
    ends = np.nonzero(df == -1)[0]
    count = 0
    for idx in range(starts.size):
        if ends[idx] - starts[idx] >= min_runsize:
            count += 1
    return count

def longest_run(tf_array):
    # Return nduration of longest consecutive run
    # Accepts true-false array with one row per year, returns array of longest durations -- one duration per year
    if np.sum(tf_array) == 0:
        return 0
    falses = np.zeros(tf_array.shape[0]).reshape((tf_array.shape[0],1))
    extended_a = np.concatenate([[0], tf_array, [0]])
    df = np.diff(extended_a)
    starts = np.nonzero(df == 1)[0]
    ends = np.nonzero(df == -1)[0]
    durations = ends - starts
    return max(durations)
    
def quarters(d, start_year, end_year, southern_hem=False):
    #Takes multi-year array and returns data reorganized into seasonal quarters
    q2 = []  # 60-151
    q3 = []  # 152-243
    q4 = []  # 244-334
    q1 = []  # 335-59
    if not southern_hem:
        jan1_idx = 365
        for year in range(start_year, end_year):
            tmp = np.concatenate((d[jan1_idx - 365 : jan1_idx - 365 + 60], d[jan1_idx + 335 : jan1_idx + 365]), axis=0)
            q1.append(tmp)
            q2.append(d[jan1_idx + 60 : jan1_idx + 152])
            q3.append(d[jan1_idx + 152 : jan1_idx + 244])
            q4.append(d[jan1_idx + 244 : jan1_idx + 335])

            jan1_idx += 365 + [0, 0][int(False and calendar.isleap(year))]
        mam_res = np.vstack(q2)
        jja_res = np.vstack(q3)
        son_res = np.vstack(q4)
        djf_res = np.vstack(q1)
    else:
        jul1_idx = 365
        for year in range(start_year, end_year):
            tmp = np.concatenate((d[jul1_idx - 365 : jul1_idx - 365 + 60], d[jul1_idx + 335 : jul1_idx + 365]), axis=0)
            q3.append(tmp)
            q4.append(d[jul1_idx + 60 : jul1_idx + 152])
            q1.append(d[jul1_idx + 152 : jul1_idx + 244])
            q2.append(d[jul1_idx + 244 : jul1_idx + 335])

            jul1_idx += 365 + [0, 0][int(False and calendar.isleap(year))]
        mam_res = np.vstack(q4)
        jja_res = np.vstack(q1)
        son_res = np.vstack(q2)
        djf_res = np.vstack(q3)
    return mam_res, jja_res, son_res, djf_res

def calibrate_component(uncalibrated_data, calibration_fxn):
    # Applies calib fxn to one data array -- generally used for one quarter's worth of data
    N = len(uncalibrated_data)
    unsorted_uncalib = [(i, idx) for idx, i in enumerate(uncalibrated_data)]
    sorted_uncalib = sorted(unsorted_uncalib)
    result = [0] * N
    for j in range(N):
        X_j = j / (N + 1)
        Y_jprime = calibration_fxn[math.floor(X_j * len(calibration_fxn))]
        jprime = math.floor(Y_jprime * (N + 1))
        result[sorted_uncalib[j][1]] = sorted_uncalib[min(len(sorted_uncalib)-1, jprime)][0]
    return result

def calibrate(uncalibrated_data, calibration_fxn):
    # Applies calibrate_component() to full all-years array, applying quarter-specific calib fxns separately
    mam = []
    jja = []
    son = []
    djf = []
    mam_idx = []
    jja_idx = []
    son_idx = []
    djf_idx = []
    for idx, i in enumerate(uncalibrated_data):
        if idx % 365 >= 60 and idx % 365 < 152:
            mam.append(uncalibrated_data[idx])
            mam_idx.append(idx)
        elif idx % 365 >= 152 and idx % 365 < 244:
            jja.append(uncalibrated_data[idx])
            jja_idx.append(idx)
        elif idx % 365 >= 244 and idx % 365 < 335:
            son.append(uncalibrated_data[idx])
            son_idx.append(idx)
        else:
            djf.append(uncalibrated_data[idx])
            djf_idx.append(idx)
    
    mam_calib = calibrate_component(np.array(mam), calibration_fxn[0])
    jja_calib = calibrate_component(np.array(jja), calibration_fxn[1])
    son_calib = calibrate_component(np.array(son), calibration_fxn[2])
    djf_calib = calibrate_component(np.array(djf), calibration_fxn[3])
    
    result = [0] * len(uncalibrated_data)
    for i in range(len(mam_idx)):
        result[mam_idx[i]] = mam_calib[i]
    for i in range(len(jja_idx)):
        result[jja_idx[i]] = jja_calib[i]
    for i in range(len(son_idx)):
        result[son_idx[i]] = son_calib[i]
    for i in range(len(djf_idx)):
        result[djf_idx[i]] = djf_calib[i]

    return np.array(result)

def ari(yeardata):
    ARI_WEIGHTS = np.array([
        0.013499274414000246,
        0.01837401239683367,
        0.026458577851440485,
        0.041341527892875755,
        0.07349604958733467,
        0.16536611157150302,
        0.6614644462860121
    ])
    def ari_7day(sevendayrain):
        return np.dot(sevendayrain, ARI_WEIGHTS)
    
    res = []
    for start_idx in range(yeardata.size-7):
        res.append(ari_7day(yeardata[start_idx:start_idx+7]))
    return res

def get_ari95(latlon):
#    ARI95 = ee.Image('users/tedwongwri/dataportal/landslide/ARI95')
#    geom = ee.Geometry.Point((latlon[1], latlon[0]))
#    res = ARI95.reduceRegion(ee.Reducer.mean(), geom, 10, 'epsg:4326').getInfo()['b1']
#    return res
    return wholeyear_percentile('ari', 95, latlon)
        

def wetbulbtemp(T, RH):
# JA Knox et al. 2017. Two simple and accurate approximations for wet-bulb
# temperature in moist conditions, with forecasting applications. Bull. Am.
# Meteorol. Soc. 98(9): 1897-1906. doi:10.1175/BAMS-D-16-0246.1
    T = T.astype(np.float64)
    rh_percent = RH.astype(np.float64)
    return T * np.arctan(0.151977 * np.sqrt(rh_percent + 8.313659)) + np.arctan(T + rh_percent) - np.arctan(rh_percent - 1.676331) + ((0.00391838 * ((rh_percent)**(3/2))) * np.arctan(0.023101 * rh_percent)) - 4.686035        


In [7]:
# Class Hazard defines two indicator fxns and beqeaths to a number of hazard-specific subclasses. The indicator fxns
# get_expectedval() and get_exceedanceprob() accept an array of meteorological data and return statistics of the
# estimated predicted expected value of the hazard magnitude, or the probability that the hazard magnitude exceeds
# a given extreme value, in the years represented in the data array. The indicator statistics are currently mean estimate
# and standard deviation of the estimate.
#
# Hazard subclasses each define a val_dist() fxn, which accepts a data array (one row per meteorological variable,
# one column for each day in the multi-year interval of interest) and returns a dict whose keys are hazard magnitudes
# and whose values are how many times (i.e., years) in the multi-year interval that value is observed in the data array.
# The dict is a frequency distribution. Both get_expectedval() and get_exceedanceprob() call on val_dist().
#
# Each hazard subclass is instantiated with the arguments necessary to calculate hazard magnitude. Some hazards
# require threshold values, some require range endmembers, and some require the minimum number of consecutive
# days to count as a run of days. All hazards also require argument extremeval, which is the threshold magnitude used
# to calculate threshold-exceedance probability.

class Hazard:
    def get_percentile(self, latlon, q):
        southern_hem = int(latlon[0] < 0)
        era_data = {}
        scenario_years = {}
        varnames = self.varname.split('+')
        for varname in varnames:
            era_data[varname] = get_observed_gee(varname, latlon, PERCENTILE_STARTYEAR, PERCENTILE_ENDYEAR, southern_hem)
        countdist = self.val_dist([ed for ed in era_data])
        return np.percentile(countdist, q)
    
    def get_expectedval(self, latlon, datasets, calib_fxns):
        # Take uncalibrated data, calibrate it, apply val_dist() to calibrated data, use resulting freq dist
        # to parameterize Dirichlet prior, take resulting vector to parameterize multinomial distribution,
        # sample from that multinomial to generate predictive distribution of freq distributions, and return statistics
        # (mean and stdev but it could be anything) of the predictive distribution.
        
        southern_hem = int(latlon[0] < 0)
        fut_mod = {}
        scenario_years = {}
        varnames = list(datasets.keys())
        for varname in varnames:
            for model in calib_fxns[self.varname].keys():
                ds = datasets[varname][model][2]
                scenario_years[model] = (datasets[varname][model][0], datasets[varname][model][1])
                fut_mod[(varname, model)] = ds
        best_models = []
        for idx in range(NUM_BEST_MODELS):
            modelplus = '+'.join([list(calib_fxns[self.varname].keys())[idx] for varname in varnames])
            best_models.append(modelplus)

        para_res = {}
        for modelplus in best_models:
            start_year = scenario_years[modelplus.split('+')[0]][0]
            end_year = scenario_years[modelplus.split('+')[0]][1]
            numbins = end_year - start_year + 1
            calib_data = []
            for idx, varname in enumerate(varnames):
                model = modelplus.split('+')[idx]
                calib_data.append(np.array(calibrate(fut_mod[(varname, model)], calib_fxns[self.varname][model])))
            countdist = self.val_dist([cd[[0,152][int(not southern_hem)]:[len(cd),-213][int(not southern_hem)]] for cd in calib_data])
            if countdist is None:
                para_res[modelplus] = None
            else:
                observed_vals = np.array(list(countdist.keys()))
                cdist = {}
                minval = observed_vals[0]
                maxval = observed_vals[-1]
                D = (maxval - minval) / (numbins - 1)
                for i in range(numbins):
                    centerval = minval + (i * D)
                    cdist[centerval] = 0
                for count in countdist:
                    for centerval in cdist:
                        if count >= centerval - (D/2) and count < centerval + (D/2):
                            cdist[centerval] += 1
                alpha = np.array(list(cdist.values())) + (1/numbins)
                res = []
                for i in range(10000):
                    dirich_samp = np.random.dirichlet(alpha, 1)
                    mult_samp = np.random.multinomial(end_year - start_year + 1, dirich_samp[0], 1)[0]
                    res.append(sum([list(cdist.keys())[j] * mult_samp[j] for j in range(len(list(cdist.keys())))]) / (end_year - start_year + 1))
                res = np.array(res)
                para_res[modelplus] = res
        result = {}
        for modelplus in best_models:
            if para_res[modelplus] is None:
                result[modelplus] = [-9999, -9999, -9999]
            else:
                result[modelplus] = [np.mean(para_res[modelplus]), np.std(para_res[modelplus]), -9999]  # -9999 is dummy to preserve format from earier version
        return result
    
    def get_exceedanceprob(self, latlon, datasets, calib_fxns):
        # Take uncalibrated data, calibrate it, apply val_dist() to calibrated data, use resulting freq dist
        # to generate analytically the mean and stdev of beta-binomial or gamma-poisson distribution of
        # threshold-exceedance probabilities. Return statistics (mean and stdev but it could be anything) of
        # the predictive distribution.
        
        def floorceiling_prob(p):
            # Ensure that probabilities are on [0, 1]
            return max(0, min(1, p))
        
        southern_hem = int(latlon[0] < 0)
        fut_mod = {}
        scenario_years = {}
        varnames = list(datasets.keys())
        for varname in varnames:
            for model in calib_fxns[self.varname].keys():
                ds = datasets[varname][model][2]
                scenario_years[model] = (datasets[varname][model][0], datasets[varname][model][1])
                fut_mod[(varname, model)] = ds
        best_models = []
        for idx in range(NUM_BEST_MODELS):
            modelplus = '+'.join([list(calib_fxns[self.varname].keys())[idx] for varname in varnames])
            best_models.append(modelplus)

        prob_res = {}
        for modelplus in best_models:
            prob_res[modelplus] = {}
            start_year = scenario_years[modelplus.split('+')[0]][0]
            end_year = scenario_years[modelplus.split('+')[0]][1]
            calib_data = []
            for idx, varname in enumerate(varnames):
                model = modelplus.split('+')[idx]
                calib_data.append(np.array(calibrate(fut_mod[(varname, model)], calib_fxns[self.varname][model])))
            countdist = self.val_dist([cd[[0,152][int(not southern_hem)]:[len(cd),-213][int(not southern_hem)]] for cd in calib_data])
            for exval in self.extremeval:
                if countdist is None:
                    prob_res[modelplus][exval] = [-9999, -9999, -9999]
                else:
                    count = sum([countdist[val] * int((val >= exval and self.exceed_is_gte) or (val <= exval and not self.exceed_is_gte)) for val in countdist])
                    N = end_year - start_year + 1
                    if self.probmodel == 'Poisson':
                        meanprob = (count + 0.5) / N
                        stdprob = np.sqrt((2 * count) + 1) / N
                    else: 
                        meanprob = ((count + 0.5) * N / (N + 1)) / N
                        stdprob = (np.sqrt((N * (count + 0.5) * (N - count + 0.5) * ((2 * N) + 1)) / ((N + 2) * (N + 1) * (N + 1)))) / N
                    prob_res[modelplus][exval] = [floorceiling_prob(meanprob), stdprob, -9999]  # -9999 is dummy to preserve format from earier version
       
        return prob_res



class Tempwave_count(Hazard):
    # Heatwaves and coldwaves
    # Number per year
    # tf_gte is True for heatwaves, False for coldwaves
    def __init__(self, hazname, varname, min_duration, threshold, tf_gte, extremeval):
        if type(threshold) == np.ndarray and threshold.size % 365 != 0:
            raise Exception('Comparison array length is not an integer multiple of 365')
        self.hazname = hazname
        self.varname = varname
        self.tf_gte = tf_gte
        self.min_duration = min_duration
        self.threshold = threshold  # May be scalar or 365-long array
        self.probmodel = 'Poisson'
        self.exceed_is_gte = True
        self.extremeval = extremeval
        
    def tf_array(self, datalist):
        data = datalist[0]
        if self.tf_gte:
            return data >= self.threshold
        else:
            return data <= self.threshold

    def val_dist(self, datalist):
        tfarray = self.tf_array(datalist)
        tfarray = tfarray.reshape(tfarray.size//365, 365)
        vals = np.apply_along_axis(count_runs, 1, tfarray, self.min_duration)
        
        result_dist = {}
        for val in np.unique(vals):
            result_dist[val] = np.sum(vals == val)
        return result_dist
    
class Tempwave_duration(Hazard):
    # Heatwaves and coldwaves
    # Duration of longest in a year
    # tf_gte is True for heatwaves, False for coldwaves
    def __init__(self, hazname, varname, threshold, tf_gte, extremeval):
        if type(threshold) == np.ndarray and threshold.size % 365 != 0:
            raise Exception('Comparison array length is not an integer multiple of 365')
        self.hazname = hazname
        self.varname = varname
        self.tf_gte = tf_gte
        self.threshold = threshold  # May be scalar or 365-long array
        self.probmodel = 'Poisson'
        self.exceed_is_gte = True
        self.extremeval = extremeval
        
    def tf_array(self, datalist):
        data = datalist[0]
        if self.tf_gte:
            return data >= self.threshold
        else:
            return data <= self.threshold

    def val_dist(self, datalist):
        tfarray = self.tf_array(datalist)
        tfarray = tfarray.reshape(tfarray.size//365, 365)
        vals = np.apply_along_axis(longest_run, 1, tfarray)
        
        result_dist = {}
        for val in np.unique(vals):
            result_dist[val] = np.sum(vals == val)
        return result_dist

class Tempwave_highlow_count(Hazard):
    # Consecutive run of days with temp within a range. Number of these runs in a year.
    def __init__(self, hazname, hightemp, lowtemp, min_duration, extremeval):
        self.hazname = hazname
        self.varname = 'tasmax+tasmin'
        self.min_duration = min_duration
        self.hightemp = hightemp
        self.lowtemp = lowtemp
        self.probmodel = 'Poisson'
        self.exceed_is_gte = True
        self.extremeval = extremeval
        
    def tf_array(self, datalist):
        data_tx = datalist[0]
        data_tn = datalist[1]
        if type(self.hightemp) in (float, int, np.float64, np.int32):
            high_threshold = self.hightemp
        else:   # type is np array
            high_threshold = np.array([])
            while high_threshold.size < data_tx.size:
                high_threshold = np.concatenate([high_threshold, self.hightemp])
        if type(self.lowtemp) in (float, int, np.float64, np.int32):
            low_threshold = self.lowtemp
        else:   # type is np array
            low_threshold = np.array([])
            while low_threshold.size < data_tn.size:
                low_threshold = np.concatenate([low_threshold, self.lowtemp])
        tf_array_tx = data_tx >= high_threshold
        tf_array_tn = data_tn >= low_threshold
        return tf_array_tx * tf_array_tn
    
    def val_dist(self, datalist):
        tfarray = self.tf_array(datalist)
        tfarray = tfarray.reshape(tfarray.size//365, 365)
        vals = np.apply_along_axis(count_runs, 1, tfarray, self.min_duration)
        result_dist = {}
        for val in np.unique(vals):
            result_dist[val] = np.sum(vals == val)
        return result_dist
    
class Tempwave_highlow_duration(Hazard):
    # Consecutive run of days with temp within a range. Duration in days of longest run in a year.
    def __init__(self, hazname, hightemp, lowtemp, min_duration, extremeval):
        self.hazname = hazname
        self.varname = 'tasmax+tasmin'
        self.min_duration = min_duration
        self.hightemp = hightemp
        self.lowtemp = lowtemp
        self.probmodel = 'Poisson'
        self.exceed_is_gte = True
        self.extremeval = extremeval
        
        
    def tf_array(self, datalist):
        data_tx = datalist[0]
        data_tn = datalist[1]
        if type(self.hightemp) in (float, int, np.float64, np.int32):
            high_threshold = self.hightemp
        else:   # type is np array
            high_threshold = np.array([])
            while high_threshold.size < data_tx.size:
                high_threshold = np.concatenate([high_threshold, self.hightemp])
        if type(self.lowtemp) in (float, int, np.float64, np.int32):
            low_threshold = self.lowtemp
        else:   # type is np array
            low_threshold = np.array([])
            while low_threshold.size < data_tn.size:
                low_threshold = np.concatenate([low_threshold, self.lowtemp])
        tf_array_tx = data_tx >= high_threshold
        tf_array_tn = data_tn >= low_threshold
        return tf_array_tx * tf_array_tn
    
    def val_dist(self, datalist):
        tfarray = self.tf_array(datalist)
        tfarray = tfarray.reshape(tfarray.size//365, 365)
        vals = np.apply_along_axis(longest_run, 1, tfarray)
        result_dist = {}
        for val in np.unique(vals):
            result_dist[val] = np.sum(vals == val)
        return result_dist

class ThresholdDays(Hazard):
    # Num days (can be nonconsecutive) meeting some criterion in a year
    def __init__(self, hazname, varname, var_threshold, want_max, extremeval):
        self.hazname = hazname
        self.varname = varname
        self.var_threshold = var_threshold
        self.want_max = want_max
        self.probmodel = 'binomial'
        self.exceed_is_gte = True
        self.extremeval = extremeval

    def val_dist(self, datalist):
        data = datalist[0]
        if data.size % 365 != 0:
            raise Exception('Data array length is not an integer multiple of 365')   
        byyear = data.reshape(data.size // 365, 365)
        
        if self.want_max:
            vals = np.sum(byyear >= self.var_threshold, axis=1)
        else:
            vals = np.sum(byyear <= self.var_threshold, axis=1)
        result_dist = {}
        for val in np.unique(vals):
            result_dist[val] = np.sum(vals == val)
        return result_dist
    
class RangeDays(Hazard):
    # Num days (can be nonconsecutive) in a year with some variable with a range
    def __init__(self, hazname, varname, var_thresholdlow, var_thresholdhigh, extremeval):
        self.hazname = hazname
        self.varname = varname
        self.var_thresholdlow = var_thresholdlow
        self.var_thresholdhigh = var_thresholdhigh
        self.probmodel = 'binomial'
        self.exceed_is_gte = True
        self.extremeval = extremeval

    def val_dist(self, datalist):
        data = datalist[0]
        byyear = data.reshape(data.size // 365, 365)
        
        vals = np.sum(np.logical_and(byyear >= self.var_thresholdlow, byyear <= self.var_thresholdhigh), axis=1)

        result_dist = {}
        for val in np.unique(vals):
            result_dist[val] = np.sum(vals == val)
        return result_dist
        



class Annual_val(Hazard):
    def __init__(self, hazname, varname, aggtype, extremeval):
        self.hazname = hazname
        self.varname = varname
        self.aggtype = aggtype
        self.probmodel = 'binomial'
        self.exceed_is_gte = True
        self.extremeval = extremeval
        
    def val_dist(self, datalist):
        data = datalist[0]
        byyear = data.reshape(data.size//365, 365)
        if self.aggtype == 'sum':
            vals = np.sum(byyear, axis=1)
        elif self.aggtype == 'mean':
            vals = np.mean(byyear, axis=1)
        elif self.aggtype == 'max':
            vals = np.max(byyear, axis=1)
        elif self.aggtype == 'min':
            vals = np.min(byyear, axis=1)
        result_dist = {}
        for val in np.unique(vals):
            result_dist[val] = np.sum(vals == val)
        return result_dist

class Seasonal_val(Hazard):
    # Sum, mean, max, or min of some variable value within a year
    def __init__(self, hazname, varname, aggtype, startdate, enddate, southern_hem, extremeval):
        self.hazname = hazname
        self.varname = varname
        self.aggtype = aggtype
        self.startdate = startdate
        self.enddate = enddate
        self.southern_hem = southern_hem
        self.probmodel = 'binomial'
        self.exceed_is_gte = True
        self.extremeval = extremeval
        
    def val_dist(self, datalist):
        data = datalist[0]
        byyear = data.reshape(data.size//365, 365)
        start_jday = d2j('1999-{0}'.format(self.startdate)) - [0, 182][int(self.southern_hem)]
        end_jday = d2j('1999-{0}'.format(self.enddate)) - [0, 182][int(self.southern_hem)]
        if end_jday < start_jday:
            end_jday += 365
        inseason_onerow = [((i >= start_jday)and(i <= end_jday)) for i in range(365)]
        inseason = np.array([inseason_onerow]*(data.size//365))
        byyear = byyear * inseason
        if self.aggtype == 'sum':
            vals = np.sum(byyear, axis=1)
        elif self.aggtype == 'mean':
            vals = np.mean(byyear, axis=1)
        elif self.aggtype == 'max':
            vals = np.max(byyear, axis=1)
        elif self.aggtype == 'min':
            vals = np.min(byyear, axis=1)
        result_dist = {}
        for val in np.unique(vals):
            result_dist[val] = np.sum(vals == val)
        #print(vals)
        return result_dist

class ARIDays(Hazard):
    # Antecedent Rainfall Index, used for landslide risk
    # Num days with heavy rainfall in the five prior days
    # Must be combined with geological instability to get landslide risk
    def __init__(self, hazname, ari95, extremeval):
        self.hazname = hazname
        self.varname = 'pr'
        self.probmodel = 'binomial'
        self.exceed_is_gte = True
        self.extremeval = extremeval

    def val_dist(self, datalist):
        data = datalist[0]
        if data.size % 365 != 0:
            raise Exception('Data array length is not an integer multiple of 365')   
        byyear = data.reshape(data.size // 365, 365)
        
        aris = np.apply_along_axis(ari, 1, byyear)
        vals = np.sum(aris >= ari95, axis=1)
        
        result_dist = {}
        for val in np.unique(vals):
            result_dist[val] = np.sum(vals == val)
        return result_dist
    
class WetbulbDays(Hazard):
    # Days in a year (can be nonconsecutive) with wetbulb temp exceeding some value
    def __init__(self, hazname, wbt_threshold, extremeval):
        self.hazname = hazname
        self.varname = 'maxwetbulb'
        self.wbt_threshold = wbt_threshold
        self.probmodel = 'binomial'
        self.exceed_is_gte = True
        self.extremeval = extremeval
    
    def val_dist(self, datalist):
        data_t = datalist[0]
        data_h = datalist[1]
        data = wetbulbtemp(data_t, data_h)
        if data.size % 365 != 0:
            raise Exception('Data array length is not an integer multiple of 365')
        byyear = data.reshape(data.size//365, 365)
        vals = np.sum(byyear >= self.wbt_threshold, axis=1)
        result_dist = {}
        for val in np.unique(vals):
            result_dist[val] = np.sum(vals == val)
        return result_dist

class DroughtSPIDays(Hazard):
    # Meteorological drought hazard based on Standardized Precipitation Index
    def __init__(self, hazname, extremeval):
        self.hazname = hazname
        self.varname = 'pr'
        self.probmodel = 'binomial'
        self.exceed_is_gte = True
        self.extremeval = extremeval
    
    def val_dist(self, datalist):
        data = datalist[0]
        if data.size % 365 != 0:
            raise Exception('Data array length is not an integer multiple of 365')
        
        t=pd.date_range(start='1980-01-01', end='{0}-12-31'.format(1980 + (data.size//365) - 1), freq='D')
        t = t[~((t.month == 2) & (t.day == 29))]
        
        try:
            droughtdays = spei.spi(pd.Series(data, index=t)).to_numpy()
        except:
            return None
        byyear = droughtdays.reshape(data.size // 365, 365)
        
        vals = np.sum(byyear <= -1.5, axis=1)
        result_dist = {}
        for val in np.unique(vals):
            result_dist[val] = np.sum(vals == val)
        return result_dist
    
class DegreeDays(Hazard):
    # Yearlong sum of positive deviations from reference temperature
    def __init__(self, hazname, reftemp, want_max, extremeval):
        self.hazname = hazname
        self.varname = 'tas'
        self.reftemp = reftemp
        self.extremeval = extremeval
        self.want_max = want_max
        self.probmodel = 'binomial'
        self.exceed_is_gte = True

    def val_dist(self, datalist):
        data = datalist[0]
        if data.size % 365 != 0:
            raise Exception('Data array length is not an integer multiple of 365')   
        byyear = data.reshape(data.size // 365, 365)
        
        if self.want_max:
            vals = np.sum(np.round(np.maximum(0, byyear - self.reftemp)), axis=1)
        else:
            vals = np.sum(np.round(np.maximum(0, (-1 * byyear) + self.reftemp)), axis=1)
        result_dist = {}
        for val in np.unique(vals):
            result_dist[val] = np.sum(vals == val)
        return result_dist

In [8]:
def get_observed_gee(varname, latlon, start_year, end_year, southern_hem=False):
    # Get ERA5 data from Google Earth Engine
    # Return numpy array in correct units, leapdays removed
    def relhum(T, Tdp):
        # relative humidity as percent on [0, 100]
        # Used only if variable of interest is hurs
        T = T.astype('float64')
        Tdp = Tdp.astype('float64')
        numerator = np.exp(17.625 * Tdp / (243.04 + Tdp))
        denominator = np.exp(17.625 * T / (243.04 + T))
        return 100 * numerator / denominator

    def get_eradata(varname, southern_hem=False):
        # Return numpy array in correct units, leapdays removed
        dataset = ee.ImageCollection("ECMWF/ERA5/DAILY")
        gee_geom = ee.Geometry.Point((latlon[1], latlon[0]))
        data_vars = dataset.select(varname).filter(ee.Filter.date('{0}-01-01'.format(start_year), '{0}-01-01'.format(end_year+ 1)))
        success = False
        while not success:
            try:
                d = data_vars.getRegion(gee_geom, 2500, 'epsg:4326').getInfo()
                success = True
            except:
                print('\nRetrying')
        result = [i[4] for i in d[1:]]
        return np.array(result)
    
    if varname == 'hurs':
        success = False
        era_dewpoint = get_eradata('dewpoint_2m_temperature')-273.15
        era_maxtemp = get_eradata('maximum_2m_air_temperature')-273.15
        hist_obs = relhum(era_maxtemp, era_dewpoint)
    elif varname == 'pr':
        hist_obs = get_eradata('total_precipitation') * 1000
    elif varname == 'tasmax':
        hist_obs = get_eradata('maximum_2m_air_temperature')-273.15
    else:    # varname == 'tasmin'
        hist_obs = get_eradata('minimum_2m_air_temperature')-273.15
    return removeLeapDays(hist_obs, start_year, end_year, southern_hem)

def get_modeled_gee(varname, scenario, model, lat, lon, southern_hem, start_year, end_year):
    # Get NEX-GDDP-CMIP5 data from Google Earth Engine
    # Return numpy array in correct units, uncalibrated, leapdays removed
    dataset = ee.ImageCollection('NASA/GDDP-CMIP6').filter(ee.Filter.eq('model', model)).filter(ee.Filter.eq('scenario', scenario))
    gee_geom = ee.Geometry.Point((lon, lat))
    if start_year >= 2015 and not (start_year==2015 and southern_hem):
        if southern_hem:
            data_vars = dataset.select(varname).filter(ee.Filter.date('{0}-07-01'.format(start_year-1), '{0}-07-01'.format(end_year)))
        else:
            data_vars = dataset.select(varname).filter(ee.Filter.date('{0}-01-01'.format(start_year), '{0}-01-01'.format(end_year+ 1)))
        result = [i[4] for i in data_vars.getRegion(gee_geom, 2500, 'epsg:4326').getInfo()[1:]]
    else:
        hist_dataset = ee.ImageCollection('NASA/GDDP-CMIP6').filter(ee.Filter.eq('model', model))
        if southern_hem:
            hist_part = hist_dataset.select(varname).filter(ee.Filter.eq('scenario', 'historical')).filter(ee.Filter.date('{0}-07-01'.format(start_year-1), '2015-01-01'))
            if end_year >= 2015:
                ssp_part = dataset.select(varname).filter(ee.Filter.eq('scenario', scenario)).filter(ee.Filter.date('2015-01-01', '{0}-07-01'.format(end_year)))
        else:
            hist_part = hist_dataset.select(varname).filter(ee.Filter.eq('scenario', 'historical')).filter(ee.Filter.date('{0}-01-01'.format(start_year), '2015-01-01'))
            if end_year >= 2015:
                ssp_part = dataset.select(varname).filter(ee.Filter.eq('scenario', scenario)).filter(ee.Filter.date('2015-01-01'.format(start_year-1), '{0}-01-01'.format(end_year+ 1)))
        hist_result = [i[4] for i in hist_part.getRegion(gee_geom, 2500, 'epsg:4326').getInfo()[1:]]
        if end_year >= 2015:
            ssp_result = [i[4] for i in ssp_part.getRegion(gee_geom, 2500, 'epsg:4326').getInfo()[1:]]
        else:
            ssp_result = []
        result = hist_result + ssp_result
    d =  VARIABLES[varname]['nex_transform'](np.array(result))
    return removeLeapDays(d, start_year, end_year, southern_hem)

In [9]:
def getdata(varnames, warmingscenario, lat, lon, best_models):
    res = {varname: {} for varname in varnames}
    for varname in varnames:
        for model in best_models[varname]:
            scenario = scenarioyears[model][warmingscenario][0]
            start_year = scenarioyears[model][warmingscenario][1] - 4
            end_year = scenarioyears[model][warmingscenario][1] + 5
            res[varname][model] = (start_year, end_year, get_modeled_gee(varname, scenario, model, lat, lon, lat < 0, start_year, end_year))
    return res

def do_locationhazard(loc_id, hazard, datasets, latlon, scenario, calib_fxns):
    lat, lon = latlon    
    return loc_id, lat, lon, hazard.hazname, scenario, hazard.get_expectedval(latlon, datasets, calib_fxns), hazard.get_exceedanceprob(latlon, datasets, calib_fxns)
    #return hazard.get_exceedanceprob(latlon, datasets, calib_fxns)

In [10]:
# This dict helps with calculation of max wetbulb temp, which is the only hazard in this project
# that is calculated with more than one modeled meteorological variable.

SYNTHVARS = {
    'maxwetbulb': {
        'nex_varnames': ['tasmax', 'hurs'],
        'era_varnames': ['maximum_2m_air_temperature', 'dewpoint_2m_temperature'],
        'nex_fxn': lambda a, b: wetbulbtemp(a,b),
        'era_fxn': lambda a, b: wetbulbtemp(a, relhum(a, b)),
        'era_transform': [lambda x: x - 273.15, lambda x: x - 273.15]
    }
}

In [11]:
# Calibration functions are arrays of four arrays, each of which is for one seasonal quarter and contains approx 3000
# numbers on [0,1]. These numbers describe the P-P plot for one meteorological variable, for historical values from ERA5
# and from a NEX-GDDP-CMIP6 model for the same historical years. This fxn reads calib fxns for three models for each location
# and stores them in a dict with location-id as keys.

def get_calibfxns(varname, cityinfo):
    cf = {loc_id: {} for loc_id in range(len(cityinfo))}
    for varname in varname.split('+'):
        with open('bmcf_{0}.txt'.format(varname), 'r') as ifile:
            lines = ifile.readlines()
            for linenum, line in enumerate(lines):
                items = [i.strip() for i in line.split('\t')]
                if linenum % 3 == 0:
                    cf[int(items[0])][varname] = {}
                cf[int(items[0])][varname][items[1]] = json.loads(items[2])
    return cf

In [12]:
# Scenario years are the years used for each warming scenario. Each model has a different 10-year scenario-year interval.
scenarioyears = {}
with open('scenarioyears_v2.csv', 'r') as ifile:
    lines = ifile.readlines()
    for line in lines:
        items = [i.strip() for i in line.split(',')]
        scenarioyears[items[0]] = {
            'baseline': ('ssp245', 2015),
            '1.5C': (items[1], int(items[2])),
            '2.0C': (items[1], int(items[3])),
            '3.0C': (items[1], int(items[4]))
        }

In [13]:
%%time
results = []
for varname in ['tasmax', 'tas', 'pr', 'maxwetbulb']:
    print()
    print(varname)
    calibfxns = get_calibfxns(varname, CITYLATLON)
    for cityname in CITYLATLON:
        lat, lon, loc_id = CITYLATLON[cityname]
        loc_id = int(loc_id)
        print(loc_id, end=' ')
        latlon = (lat, lon)
        if varname == 'tasmax':
            tasmax90 = yearextreme_percentile('tasmax', 90, latlon, True)
            HAZARDS = [
                Tempwave_count('Heatwavecount tmax 90pctl 3plus', 'tasmax', 3, tasmax90, True, [1, 3, 5]),
                Tempwave_duration('Heatwaveduration tmax 90pctl', 'tasmax', tasmax90, True, [20, 30, 40]),
                ThresholdDays('Days warmer than 35', 'tasmax', 35, True, [10, 20, 30]),
                ThresholdDays('Days warmer than 40', 'tasmax', 40, True, [10, 20, 30]),
                ThresholdDays('Days warmer than than 95th pctle yearlong', 'tasmax', tasmax90, True, [60, 70, 80]),
                Annual_val('Hottest annual temp', 'tasmax', 'max', [35, 40, 45]),
            ]
        elif varname == 'pr':
            pr90 = yearextreme_percentile('pr', 90, latlon, True)
            ari95 = get_ari95(latlon)
            HAZARDS = [
                Annual_val('Highest daily precip', 'pr', 'max', [500, 1000, 2000]),
                ThresholdDays('Days precip gte 90th pctl', 'pr', pr90, True, [20, 30, 40]),
                ARIDays('ARI days gte ari95', ari95, [5, 10, 20]),
                DroughtSPIDays('Drought days SPI lte -2', [100, 140, 180])
            ]
        elif varname == 'tas':
            HAZARDS = [
                DegreeDays('CDD21', 21, True, [2000, 3000, 4000]),
                RangeDays('arbovirus temp', 'tas', 26, 29, [30, 60, 90]), #https://journals.plos.org/plosntds/article?id=10.1371/journal.pntd.0005568
                RangeDays('malaria temp', 'tas', 22.9, 27.8, [30, 60, 90]), # https://malariajournal.biomedcentral.com/articles/10.1186/s12936-020-03224-6
            ]
        elif varname == 'maxwetbulb':
            HAZARDS = [
                WetbulbDays('Wetbulb days gte 31', 31, [10, 25, 30]),
            ]
        
        warming_scenarios = ['baseline','1.5C', '2.0C', '3.0C']
        varnames = []
        for hazard in HAZARDS:
            if hazard.varname == varname:
                for scenario in warming_scenarios:
                    if varname in SYNTHVARS:
                        varnames += SYNTHVARS[varname]['nex_varnames']
                    else:
                        varnames += hazard.varname.split('+')
        varnames = list(set(varnames))
        allds = {scenario: getdata(varnames, scenario, lat, lon, {vn: list(calibfxns[loc_id][varname].keys()) for vn in varnames})
                    for scenario in warming_scenarios}
        for hazard in HAZARDS:
            if hazard.varname == varname:
                for scenario in warming_scenarios:
                    if varname in SYNTHVARS:
                        varnames = SYNTHVARS[varname]['nex_varnames']
                    else:
                        varnames = hazard.varname.split('+')
                    datasets = allds[scenario]
                    res = do_locationhazard(loc_id, hazard, datasets, (lat, lon), scenario, calibfxns[loc_id])
                    rlocid, rlat, rlon, rhazname, rscenario, rev, rprob = res
                    with open('RESULTS.csv', 'a') as ifile:
                        for rank, rmod in enumerate(list(rev.keys())):
                            if len(rmod.split('+')) > 1:
                                displaymod = rmod.split('+')[0]
                            else:
                                displaymod = rmod
                            ifile.write('{0},{1},{2},{3},{4},'.format(rlocid, rlat, rlon, rhazname, rscenario))
                            ifile.write('{0},{1},{2},{3},{4}'.format(displaymod, rank + 1, rev[rmod][0], rev[rmod][1], rev[rmod][2]))
                            for rexval in rprob[rmod]:
                                ifile.write(',{0},{1},{2},{3}'.format(rexval, rprob[rmod][rexval][0], rprob[rmod][rexval][1], rprob[rmod][rexval][2]))
                            ifile.write('\n')


pr
0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 CPU times: total: 1h 40min 53s
Wall time: 2h 45min 34s
